In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import tensorflow as tf
from scipy.stats import pearsonr

In [35]:
import sys
sys.path.append('..')
from dataset import batcher

In [46]:
MODEL_DIR = '../saved_models/'
LR_RATE = 0.001
OPT = tf.keras.optimizers.Adam(learning_rate=LR_RATE)
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

In [63]:
# Load model
experiments = ['imagery_sample_vgg_regression']

def load_model_from_dir(model_name):
    model_path = MODEL_DIR + model_name + '.h5'
    print(model_path)
    model = tf.keras.models.load_model(model_path, compile=False)
    
    return model

In [64]:
# Get test split
test_batcher = batcher.Batcher(bucket=False, shuffle=False, split='test').get_dataset()

In [65]:
# Get consumption predictions
trained_model = load_model_from_dir(experiments[0])
trained_model.compile(
            loss=LOSS,
            optimizer=OPT,
            metrics=METRICS,
        )
consumption_predicts = trained_model.predict(test_batcher, steps=652)

../saved_models/imagery_sample_vgg_regression.h5


2023-01-28 15:30:29.362384: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


NotFoundError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "/Users/omshah/opt/anaconda3/envs/pypoverty38/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/omshah/opt/anaconda3/envs/pypoverty38/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/omshah/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/omshah/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/Users/omshah/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/omshah/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/omshah/opt/anaconda3/envs/p

In [60]:
# Find r-squared
r2 = pearsonr(x, y)

NameError: name 'x' is not defined

In [ ]:
# Plot predictions